# Configuration Variables

In [ ]:
ENABLE_COLAB=True
USE_GPU=True
GENERATE_PROFILE = False

#Set MAX_FILE_TO_LOAD between 1 and 13 only (no error checking outside of that)
MAX_FILE_TO_LOAD=13
TWEETS_FOR_ANALYSIS=60000

ENABLE_PLOTS=True

CONVERT_COLUMNS=True
ENABLE_DATA_MANIP=False

#Data Extraction Config
DATA_START_YEAR=2016
DATA_END_YEAR=2017
MAX_TWEETS_PER_MONTH=1500

NUMBER_TOPICS=40
MAX_TWEET_ANALYZE_PER_TOPIC=4

#Data Analyzed Output
CSV_OUTPUT_ROOT="/content/gdrive/MyDrive/ML1000/Project_3/"
CSV_OUTPUT_FILENAME="tweetAnalysisOutputMike.csv"
CSV_INPUT_FILENAME="tweetAnalysisOutputMike_classified.csv"

#Classification Setup
CLASSIFIED_SETUP=False

In [ ]:
if ENABLE_COLAB:
  DATA_FILE_ROOT_PATH='/content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/'
  PANDA_PROFILE_OUTPUT_ROOT='/content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/'

  SAVE_MODEL_ROOT='/content/gdrive/MyDrive/Colab Notebooks/ML1000_Project_3/'
else:
  DATA_FILE_ROOT_PATH='./data/'
  PANDA_PROFILE_OUTPUT_ROOT='./data/'

  SAVE_MODEL_ROOT='./model_saves/'


# Environment

In [ ]:
if ENABLE_COLAB:
  !pip install pycaret -q
  !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip -q
  !pip install matplotlib -q
  !pip install pandasql -q
else:
  display('Google Colab not enabled')

     |████████████████████████████████| 266 kB 5.2 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 16.9 MB 89 kB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 1.7 MB 31.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 271 kB 56.0 MB/s 
     |████████████████████████████████| 2.0 MB 51.7 MB/s 
     |████████████████████████████████| 113 kB 68.4 MB/s 
     |████████████████████████████████| 1.3 MB 47.0 MB/s 
     |████████████████████████████████| 167 kB 58.2 MB/s 
     |████████████████████████████████| 6.8 MB 40.3 MB/s 
     |████████████████████████████████| 261 kB 64.2 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 3.1 MB 31.0 MB/s 
     |███████████████████

In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import spacy
import pandasql as ps

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

%matplotlib inline

import json


!python -m spacy download en_core_web_sm 
!python -m textblob.download_corpora

     |████████████████████████████████| 12.0 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [1]:
if ENABLE_COLAB:
  from pycaret.utils import enable_colab
  enable_colab()
else:
  display('Google Colab not enabled')

NameError: ignored

In [ ]:
if ENABLE_COLAB: 
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
else:
  display('Google Colab not enabled')

Mounted at /content/gdrive


# Data Loading and Display

In [ ]:
FILE_NAME_BASE = "IRAhandle_tweets_"

index = 1
while index <= MAX_FILE_TO_LOAD:
    #read file
    FILE_TO_BE_READ = DATA_FILE_ROOT_PATH + FILE_NAME_BASE + str(index) + ".csv"
    print ("Reading file..." + FILE_TO_BE_READ)
    tempDF = pd.read_csv(FILE_TO_BE_READ, low_memory=False)

    if index == 1:
      data_orig = tempDF
    else:
      data_orig = data_orig.append(tempDF)
    
    index += 1
data_orig = data_orig.reset_index()
data_orig.shape

Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_1.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_2.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_3.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_4.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_5.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_6.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_7.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_8.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_Troll_Tweet_Data/IRAhandle_tweets_9.csv
Reading file.../content/gdrive/MyDrive/ML1000/Project_3/Russian_

(2946207, 22)

In [ ]:
data_orig.head(5)

,index,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [ ]:
data_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 22 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   index               int64 
 1   external_author_id  object
 2   author              object
 3   content             object
 4   region              object
 5   language            object
 6   publish_date        object
 7   harvested_date      object
 8   following           int64 
 9   followers           int64 
 10  updates             int64 
 11  post_type           object
 12  account_type        object
 13  retweet             int64 
 14  account_category    object
 15  new_june_2018       int64 
 16  alt_external_id     object
 17  tweet_id            int64 
 18  article_url         object
 19  tco1_step1          object
 20  tco2_step1          object
 21  tco3_step1          object
dtypes: int64(7), object(15)
memory usage: 494.5+ MB


# Data Manipulation

In [ ]:
if CONVERT_COLUMNS:
  #data_orig['harvested_date'] = pd.to_datetime(data_orig['harvested_date'])
  data_orig['publish_date'] = pd.to_datetime(data_orig['publish_date'])
  data_orig['tweet_id'] = data_orig['tweet_id'].astype('string')
  data_orig.info()
  #data_orig.groupby([data_orig["publish_date"].dt.year, data_orig["publish_date"].dt.month]).count().plot(kind="bar")
else:
  print("CONVERT_COLUMNS is not enabled")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 22 columns):
 #   Column              Dtype         
---  ------              -----         
 0   index               int64         
 1   external_author_id  object        
 2   author              object        
 3   content             object        
 4   region              object        
 5   language            object        
 6   publish_date        datetime64[ns]
 7   harvested_date      object        
 8   following           int64         
 9   followers           int64         
 10  updates             int64         
 11  post_type           object        
 12  account_type        object        
 13  retweet             int64         
 14  account_category    object        
 15  new_june_2018       int64         
 16  alt_external_id     object        
 17  tweet_id            string        
 18  article_url         object        
 19  tco1_step1          object        
 20  tc

In [ ]:
if GENERATE_PROFILE:
  profiles = ProfileReport(data_clean, title="Russian Troll Tweets", html={'style': {'full_width': True}})
  profiles.to_file(output_file= PANDA_PROFILE_OUTPUT_ROOT + "Russian_Troll_Tweets_Data_Profile.html")

else:
  display("Generate profile is off")

'Generate profile is off'

# Data Selection

In [ ]:
data_clean = data_orig

In [ ]:
display("1:" + str(data_clean.shape))
data_clean = data_clean.loc[
                            (data_clean['account_type'] == 'Right') & 
                            (data_clean['post_type'] != 'RETWEET') &
                            (data_clean['post_type'] != 'QUOTE_TWEET') &
                            (data_clean['language'] == 'English')
                            ]
display("2:" + str(data_clean.shape))
data_clean = data_clean.drop_duplicates(subset=['tweet_id'], keep='last')
display("3:" + str(data_clean.shape))

data_clean2 = data_clean[['tweet_id','content']]
data_clean2 = data_clean2.drop_duplicates(subset=['tweet_id','content'], keep='last')
display("4:" + str(data_clean2.shape))

'1:(2946207, 22)'

'2:(399850, 22)'

'3:(398468, 22)'

'4:(398468, 2)'

In [ ]:
#Extract X number of tweets per year/month combo

#Need a formatted data frame that is empty
data_sampled = data_clean.loc[[1], :]
data_sampled.drop(data_sampled.index, inplace=True)
display("Empty dataframe: " + str(data_sampled.shape))

indexYear = DATA_START_YEAR

while indexYear <= DATA_END_YEAR:
  indexMonth = 1
  while indexMonth <= 12:
    display("Working on Year-Month: " + str(indexYear) + "-" + str(indexMonth))
    #get data from frame
    tempFrame = data_clean.loc[
                              (data_clean['publish_date'].dt.year == indexYear)  &
                              (data_clean['publish_date'].dt.month == indexMonth)
                             ]
    #sample amount per month
    if len(tempFrame.index > 0):
      tempFrame = tempFrame.sample(min(MAX_TWEETS_PER_MONTH,len(tempFrame.index)))

    #append data
    data_sampled = data_sampled.append(tempFrame)
    display("data_sampled size: " + str(data_sampled.shape))
    indexMonth += 1
  indexYear += 1
data_sampled.info()

'Empty dataframe: (0, 22)'

'Working on Year-Month: 2016-1'

'data_sampled size: (1500, 22)'

'Working on Year-Month: 2016-2'

'data_sampled size: (3000, 22)'

...

'Working on Year-Month: 2017-11'

'data_sampled size: (34500, 22)'

'Working on Year-Month: 2017-12'

'data_sampled size: (35861, 22)'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35861 entries, 2291242 to 673679
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   index               35861 non-null  int64         
 1   external_author_id  35861 non-null  object        
 2   author              35861 non-null  object        
 3   content             35861 non-null  object        
 4   region              35755 non-null  object        
 5   language            35861 non-null  object        
 6   publish_date        35861 non-null  datetime64[ns]
 7   harvested_date      35861 non-null  object        
 8   following           35861 non-null  int64         
 9   followers           35861 non-null  int64         
 10  updates             35861 non-null  int64         
 11  post_type           0 non-null      object        
 12  account_type        35861 non-null  object        
 13  retweet             35861 non-null  int

# Data Setup

In [ ]:
if CLASSIFIED_SETUP:
  #Only want the movies we have classified
  #Load classified data
  classifiedDF = pd.read_csv(CSV_OUTPUT_ROOT + CSV_INPUT_FILENAME, low_memory=False)
  classifiedDF['tweet_id'] = classifiedDF['tweet_id'].astype('string')
  classifiedDF['tweet_id'] = classifiedDF['tweet_id'].str[2:20]
  display("1: " + str(classifiedDF.shape))
  
  data_analysis = data_clean
  display("2: " + str(data_analysis.shape))
  tempDF = classifiedDF[['tweet_id','manualCategory', 'manualCategoryID']]
  display(str(tempDF.head(1)))
  display("3: " + str(tempDF.shape))
  data_analysis = pd.merge(data_analysis, tempDF, how='inner', on='tweet_id')
  data_analysis['manualCategory'] = data_analysis['manualCategory'].astype('string')
  data_analysis = data_analysis[['tweet_id', 'content', 'manualCategoryID']]
else:
  #this is the first run
  data_analysis = data_sampled


data_analysis.reset_index(inplace=True, drop=True)
#display(data_analysis.shape)
#data_analysis = data_analysis.drop_duplicates(subset=['tweet_id'])
display(data_analysis.shape)
data_analysis.info()

(35861, 22)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35861 entries, 0 to 35860
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   index               35861 non-null  int64         
 1   external_author_id  35861 non-null  object        
 2   author              35861 non-null  object        
 3   content             35861 non-null  object        
 4   region              35755 non-null  object        
 5   language            35861 non-null  object        
 6   publish_date        35861 non-null  datetime64[ns]
 7   harvested_date      35861 non-null  object        
 8   following           35861 non-null  int64         
 9   followers           35861 non-null  int64         
 10  updates             35861 non-null  int64         
 11  post_type           0 non-null      object        
 12  account_type        35861 non-null  object        
 13  retweet             35861 non-null  int64     

In [ ]:
from pycaret.nlp import *



exp_p3 = setup(data = data_analysis, 
                      target = 'content', 
#                      ignore_features = ([
#                                        'index', 
#                                        'external_author_id', 
#                                        'author', 
#                                        #'content', 
#                                        'region',
#                                        'language',
#                                        'publish_date',
#                                        'harvested_date',
#                                        'following',
#                                        'followers',
#                                        'updates',
#                                        'post_type',
#                                        'account_type',
#                                        'retweet',
#                                        'account_category',
#                                        'new_june_2018',
#                                        'alt-external_id',
#                                        'tweet_id',
#                                        'article_url',
#                                        'tco1_step1',
#                                        'tco2_step1',
#                                        'tco3_step1',
#                                        'manualCategory',
#                                        #'manualCategoryID'
#                                        ]),
                   custom_stopwords = ['co', 'say', 'https', 'http', 'get', 'new', 'go', 'test', 'call', 'try', 'take', 'want', #Stopwords for Plot2
                                       'man', 'good', 'know', 'see', 'make', 'break', 'man', 'woman', 'thing', 'great', 'top', 'thank' #Stopwords for Plot3 
                                       
                                       ],
                   #log_experiment = True, experiment_name = 'kiva1'
               
               session_id = 123
               )

Description,Value
session_id,123
Documents,35861
Vocab Size,25187
Custom Stopwords,True


In [ ]:
if CLASSIFIED_SETUP:
  lda = tune_model(model = 'lda', 
                   estimator='knn', 
                   supervised_target = 'manualCategoryID', 
                   multi_core=True)
  tuned_lda = lda
else:
  lda = create_model('lda', multi_core=True, num_topics=NUMBER_TOPICS)
  untuned_lda = lda

In [ ]:
print(lda)

LdaModel(num_terms=25187, num_topics=30, decay=0.5, chunksize=100)


In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'topic_distribution')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'frequency')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'distribution')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'bigram')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'trigram')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'sentiment')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
#couldn't get this plot to work. Commenting out

#if ENABLE_PLOTS:
#  plot_model(lda, plot = 'pos')
#else:
#  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'tsne')
else:
  display("ENABLE_PLOTS not enabled")

ValueError: ignored

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'topic_model')
else:
  display("ENABLE_PLOTS not enabled")

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.476242 -0.003184       1        1  4.069927
4      0.252333  0.413267       2        1  3.474941
16     0.332483  0.294886       3        1  2.916151
36    -0.449443 -0.098616       4        1  2.687245
13     0.297930 -0.341064       5        1  2.670448
32     0.065573 -0.435314       6        1  2.656204
28    -0.230629 -0.393049       7        1  2.569870
38     0.337807 -0.226597       8        1  2.535155
10    -0.071586  0.424247       9        1  2.508225
1      0.173852 -0.401181      10        1  2.502961
37    -0.413170  0.138757      11        1  2.489772
29     0.050722 -0.244626      12        1  2.487498
18     0.446902  0.116219      13        1  2.480680
33     0.069997  0.158340      14        1  2.480630
11     0.316392 -0.086658      15        1  2.480096
12    -0.084515 -0.229491      16        1  2.467215
19    -0.310121  0.326603      17        1  2.461810
27     0.312000  0.099754      18        1  2.460717
31    -0.356624 -0.166806      19        1  2.453502
26    -0.425899  0.028205      20        1  2.432737
20    -0.095823  0.216190      21        1  2.429512
8     -0.125597 -0.374143      22        1  2.426888
34    -0.346496 -0.272912      23        1  2.408075
6      0.163098  0.042595      24        1  2.391235
14     0.032984  0.428838      25        1  2.390042
21     0.411131 -0.133357      26        1  2.375388
0      0.152938 -0.115753      27        1  2.373269
5     -0.348735  0.214352      28        1  2.362403
39    -0.161239 -0.081589      29        1  2.355439
30     0.081476  0.299788      30        1  2.348858
35     0.148595  0.352474      31        1  2.344317
9     -0.186521  0.286239      32        1  2.338991
7     -0.179458  0.386014      33        1  2.313909
23     0.187055 -0.260212      34        1  2.305322
24     0.298188  0.198979      35        1  2.289164
25    -0.233029 -0.251795      36        1  2.286230
22    -0.034187 -0.413775      37        1  2.284849
2     -0.301638 -0.015494      38        1  2.278678
3     -0.041961  0.016119      39        1  2.237531
17    -0.211026  0.103752      40        1  2.174119, topic_info=                              Term         Freq  ...  logprob  loglift
86                           trump  2914.000000  ...  30.0000  30.0000
12206                      fakenew   963.000000  ...  29.0000  29.0000
174                         people   874.000000  ...  28.0000  28.0000
410                        liberal   761.000000  ...  27.0000  27.0000
592                          video   707.000000  ...  26.0000  26.0000
...                            ...          ...  ...      ...      ...
2217                   opportunity    14.864276  ...  -5.4343   3.7726
21389                          ell    13.734853  ...  -5.5133   3.7682
2780                          kind    12.357965  ...  -5.6189   3.7617
10659                           ex    12.079839  ...  -5.6417   3.7602
24073  native_american_crazy_bitch    38.601986  ...  -4.4799   2.6114

[1235 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
4311      31  0.983325        able
608       12  0.972544    abortion
2200      36  0.970319  absolutely
1156      35  0.987941       abuse
6420      26  0.941998      abuser
...      ...       ...         ...
245       16  0.986008    year_old
4653      34  0.954811        yell
711       37  0.989488         yet
745       30  0.987920       young
25022     14  0.963662      zaogko

[1206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 5, 17, 37, 14, 33, 29, 39, 11, 2, 38, 30, 19, 34, 12, 13, 20, 28, 32, 27, 21, 9, 35, 7, 15, 22, 1, 6, 40, 31, 36, 10, 8, 24, 25, 26, 23, 3, 4, 18])

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'wordcloud')
else:
  display("ENABLE_PLOTS not enabled")

In [ ]:
if ENABLE_PLOTS:
  plot_model(lda, plot = 'umap')
else:
  display("ENABLE_PLOTS not enabled")

# Setup part two

In [ ]:
classifiedDF = pd.read_csv(CSV_OUTPUT_ROOT + CSV_INPUT_FILENAME, low_memory=False)
display(classifiedDF.info())
classifiedDF.head()

# Scratchpad

In [ ]:
assignedFrame = assign_model(lda)
assignedFrame.shape

In [ ]:
assignedFrame.reset_index(inplace=True)
display(assignedFrame.info())
display(assignedFrame.head())

In [ ]:
display(assignedFrame.shape)
tempDF = assignedFrame
display(tempDF.shape)

tempDF = tempDF.drop_duplicates(subset='tweet_id')
display(tempDF.shape)

#tempDF.drop_duplicates()

tempDF = assignedFrame[['tweet_id','content']]
display(tempDF.shape)

tempDF = tempDF.drop_duplicates()
display(tempDF.shape)

In [ ]:
#Need a formatted data frame that is empty
data_topic_output = assignedFrame.loc[[1], :]
data_topic_output.drop(data_topic_output.index, inplace=True)
display("data_topic_output size: " + str(data_topic_output.shape) + " Should be no rows")

topicIndex = 0 


#loop through number of topics
while topicIndex < NUMBER_TOPICS:
  #display("Analysing Topic " + str(topicIndex) + "...")
  #pull data for topic
  tempDF = assignedFrame.loc[(assignedFrame.Dominant_Topic == "Topic " + str(topicIndex))]
  #display(tempDF.shape)
  #sort by ranking
  tempDF = tempDF.sort_values(by=['Topic_' + str(topicIndex)], ascending=False)

  #take top X for each topic and append to dataframe
  data_topic_output = data_topic_output.append(tempDF.head(MAX_TWEET_ANALYZE_PER_TOPIC))
  
  topicIndex += 1

#output to CSV
data_topic_output.reset_index(inplace=True, drop=True)

#rename content column
data_topic_output.rename(columns={'content': 'content_parsed'}, inplace=True)
display(data_topic_output.shape)
#display(data_topic_output.info())

#merge to original load for unaltered content column, collapse to 2 columnns (all that is needed)
origContentColumn = data_analysis[['tweet_id', 'content']]
display("origContentColumn shape: " + str(origContentColumn.shape))

origContentColumn = origContentColumn.drop_duplicates()
display("origContentColumn shape: " + str(origContentColumn.shape))

csvOutput = pd.merge(data_topic_output, origContentColumn, how='inner', on='tweet_id')

display("csvOutput shape: " + str(csvOutput.shape))
#display(csvOutput.info())

csvOutput = csvOutput[['tweet_id', 'content_parsed', 'content', 'language', 'account_type', 'publish_date', 'post_type', 'retweet', 'Dominant_Topic', 'Perc_Dominant_Topic',
                       'Topic_0', 'Topic_1', 'Topic_2', 'Topic_3', 'Topic_4', 'Topic_5', 'Topic_6', 'Topic_7', 'Topic_8', 'Topic_9', 'Topic_10',
                       'Topic_11', 'Topic_12', 'Topic_13', 'Topic_14', 'Topic_15', 'Topic_16', 'Topic_17', 'Topic_18', 'Topic_19', 'Topic_20',
                       'Topic_21', 'Topic_22', 'Topic_23', 'Topic_24', 'Topic_25', 'Topic_26', 'Topic_27', 'Topic_28', 'Topic_29'
                       ]]
csvOutput['tweet_id'] = csvOutput['tweet_id'].astype('string')
csvOutput.head()
csvOutput.to_csv(CSV_OUTPUT_ROOT + CSV_OUTPUT_FILENAME)

In [ ]:
csvOutput.head()

In [ ]:
csvOutputGroup = csvOutput.groupby('tweet_id')
csvOutputGroup.filter(lambda x: len(x) > 1)

In [ ]:
dataAnalysisGroup = data_sampled
dataAnalysisGroup = dataAnalysisGroup.sort_values(by=['tweet_id'], ascending=False)
dataAnalysisGroup = dataAnalysisGroup.groupby('tweet_id')
#dataAnalysisGroup.head(5)
tempDF = dataAnalysisGroup.filter(lambda x: len(x) > 1)
tempDF.head()

In [ ]:
data_sampled_test = data_sampled
#data_sampled_test = data_sampled_test[['tweet_id']]
display(data_sampled_test.shape)
data_sampled_test = data_sampled_test.drop_duplicates(subset=['tweet_id'], keep='last')
display(data_sampled_test.shape)


#data_sampled_test = data_sampled_test.groupby('tweet_id').size().to_frame('groupCount')
#dataAnalysisGroup.head(5)
#tempDF = data_sampled_test.filter(lambda x: len(x) > 1)
#tempDF.head()
#data_sampled_test.head()